In [54]:
import pandas as pd

In [55]:
import music21 as m21

In [56]:
b = m21.corpus.parse('bwv66.6')

In [57]:
score_info = []
for i, part in enumerate(b.parts):
    for obj in part.flat.getElementsByClass(m21.note.GeneralNote):
        d = {
                'part' : i,
                'id' : obj.id,
                'pitch' : [],
                'score pos' : obj.offset,
                'duration' : obj.duration.quarterLength
            }
        if isinstance(obj, m21.note.Note):
            d['pitch'] = [obj.pitch.midi]
        elif isinstance(obj, m21.note.Chord):
            print("chord")
        score_info.append(d)

df = pd.DataFrame(score_info)
df[df['part'] == 2].head()
a = 0.5
b = 2.0
df_new = df[(df['part'] == 2) & (df['score pos'] > a) & (df['score pos'] < b)]

In [63]:
for i, row in df_new.iterrows():
    print(row.duration)

1.0
